## Problem 54: Poker hands

The file, https://projecteuler.net/project/resources/p054_poker.txt, contains one-thousand random hands dealt to two players. Each line of the file contains ten cards (separated by a single space): the first five are Player 1's cards and the last five are Player 2's cards. You can assume that all hands are valid (no invalid characters or repeated cards), each player's hand is in no specific order, and in each hand there is a clear winner.

How many hands does Player 1 win?

## Answer

The idea for anwersing is very clear: first, we should seperate player 1's cards and player 2's cards, then arrange their cards in order, then determine how 'big' their cards are, and finaly compare them for each hand and count the times when player 1 win.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# read_data

url = "https://projecteuler.net/project/resources/p054_poker.txt"
hands = pd.read_csv(url, header = None)
card_num = range(0, 9)
hands = pd.concat([hands[0].str.split(' ', expand=True)], axis = 1, names = card_num)
hands

,0,1,2,3,4,5,6,7,8,9
0,8C,TS,KC,9H,4S,7D,2S,5D,3S,AC
1,5C,AD,5D,AC,9C,7C,5H,8D,TD,KS
2,3H,7H,6S,KC,JS,QH,TD,JC,2D,8S
3,TH,8H,5C,QS,TC,9H,4D,JC,KS,JS
4,7C,5H,KC,QH,JD,AS,KH,4C,AD,4S
...,...,...,...,...,...,...,...,...,...,...
995,3S,AD,9H,JC,6D,JD,AS,KH,6S,JH
996,AD,3D,TS,KS,7H,JH,2D,JS,QD,AC
997,9C,JD,7C,6D,TC,6H,6C,JC,3D,3S
998,QC,KC,3S,JC,KD,2C,8D,AH,QS,TS


Firstly, we seperate each player's card arrange the card in order. To achieve this, I transform T, J, Q, K and A into 10, 11, 12, 13 and 14. There are two barely independent determinant in ranking players' cards: value and suit, so I store them in seperate list. The result show how I record information for player 1's cards in first hand.

In [3]:
# arrang_cards_fcn

def rep_tjqka(val):
    letters = ["T", "J", "Q", "K", "A"]
    v = 9
    for i in letters:
        v = v + 1
        while i in val:
            val[val.index(i)] = v 
    
    return val

def arrang_card(cards ,a_b):
    if a_b == "a":
        start = 0
    elif a_b == "b":
        start = 5
    
    val = []
    suit = []
    for i in range(start, start + 5):
        temp = cards[i].str.split('').tolist()
        val.append(temp[0][1])
        suit.append(temp[0][2]) 
    
    val = sorted(list(map(int, rep_tjqka(val))))
        
    return val, suit

arrang_card(hands[0:1], "a")

([4, 8, 9, 10, 13], ['C', 'S', 'C', 'H', 'S'])

Then, I need a function to distinguish cards' pattern (e.g. flush, straight, etc.). To achieve this, I define several auxiliary functions. By entering the value and suit, my `find_pattern` function can tell you which pattern the card is, and also return necessary caches needed for comparasion (e.g. value of pairs).

In [4]:
# find_pattern_fcn

def check_pairs(val):
    pairs = []
    for i in range(2, 15):
        if val.count(i) == 2:
            pairs.append(i)
        
    return sorted(pairs)

def check_three(val):
    three = 0
    for i in range(2, 15):
        if val.count(i) == 3:
            three = i
    
    return three

def check_four(val):
    four = 0
    for i in range(2, 15):
        if val.count(i) == 4:
            four = i
    
    return four

def check_straight(val):
    straight = 0
    check = len(check_pairs(val)) == 0 and check_three(val) == 0 and check_four(val) == 0
    if check:
        if val[-1] - val[0] == 4:
            straight = val[-1]
    
    return straight

def check_flush(suit):
    flush = 0
    if len(set(suit)) == 1:
        flush = 1
        
    return flush                 

def find_pattern(val, suit):
    pairs = check_pairs(val)
    three = check_three(val)
    four = check_four(val)
    straight = check_straight(val)
    flush = check_flush(suit)
    
    if straight and flush:
        pattern = 1
    elif four:
        pattern = 2
    elif len(pairs) == 1 and three:
        pattern = 3
    elif flush:
        pattern = 4
    elif straight:
        pattern = 5
    elif three:
        pattern = 6
    elif len(pairs) == 2:
        pattern = 7
    elif len(pairs) == 1:
        pattern = 8
    else:
        pattern = 9
    
    cache = [pairs, three, four, straight, flush]
    
    return cache, pattern

Next, I construct a function to do the comparasion between two players' cards. When patterns are different, it is simple to rank; but if two cards have same pattern, we need further evaluation. I create a `compare_high` function to deal with this problem.

In [5]:
# compare_fcn

def high(a, b, a2=0, b2=0, a3 = 0, b3 = 0):
    if a > b:
        r = 1
    elif a < b:
        r = 0
    elif a2 > b2:
        r = 1
    elif a2 < b2:
        r = 0
    elif a3 > b3:
        r = 1
    else:
        r = 0
        
    return r

def compare_high(a, b, p, v_a, v_b):
    if p == 1:
        r = high(a[3], b[3])
    elif p == 2:
        r = high(a[2], b[2])
    elif p == 3:
        r = high(a[1], b[1], a[0][0], b[0][0])
    elif p == 4:
        r = high(max(v_a), max(v_b))
    elif p == 5:
        r = high(a[3], b[3])
    elif p == 6:
        r = high(a[1], b[1])
    elif p == 7:
        a3 = list(set(v_a))
        a3.remove(a[0][1])
        a3.remove(a[0][0])
        b3 = list(set(v_b))
        b3.remove(b[0][1])
        b3.remove(b[0][0])
        r = high(a[0][1], b[0][1], a[0][0], b[0][0], a3, b3)
    elif p == 8:
        a2 = list(set(v_a))
        a2.remove(a[0][0])
        b2 = list(set(v_b))
        b2.remove(b[0][0])
        r = high(a[0][0], b[0][0], max(a2), max(b2))
    elif p == 9:
        r = high(max(v_a), max(v_b))
        
    return r
        
def compare(val_a, suit_a, val_b, suit_b):
    cache_a, pattern_a = find_pattern(val_a, suit_a)
    cache_b, pattern_b = find_pattern(val_b, suit_b)
    if pattern_a < pattern_b:
        result = 1
    elif pattern_a > pattern_b:
        result = 0
    else:
        result = compare_high(cache_a, cache_b, pattern_a, val_a, val_b)
            
    return result, cache_a, cache_b, pattern_a, pattern_b

The final step is just to wrap the above functions up into a `main` function and do a loop to compare the 1000 hands' cards in dataset. The result is printed below.

In [6]:
# final step

def main(hands):
    a_win = 0
    for i in range(0, len(hands)):       
        val_a, suit_a = arrang_card(hands[i:(i + 1)], "a")
        val_b, suit_b = arrang_card(hands[i:(i + 1)], "b")
        a_win = a_win + compare(val_a, suit_a, val_b, suit_b)[0]
    
    return a_win

main(hands)

376

## An elegant solution

I would like to share one brilliant solution, which solve the problem within 20 lines! The author's code does the similar thing in more concise way: basicly it creates a **score** list consists of *pattern score* and *cards value* for each player's cards; when comparing, first compare *pattern score*, if *pattern scores* are tie, then compare the high card (which stored in order in *cards value*, the second element of **score** list), which can be implemented just by using `score_list1 > score_list2` (if 1 win, return `True`, otherwise return `False`)

In [7]:
from collections import Counter

hands = (line.split() for line in open('p054_poker.txt'))

values = {r:i for i,r in enumerate('23456789TJQKA', 2)}
straights = [(v, v-1, v-2, v-3, v-4) for v in range(14, 5, -1)] + [(14, 5, 4, 3, 2)]
ranks = [(1,1,1,1,1),(2,1,1,1),(2,2,1),(3,1,1),(),(),(3,2),(4,1)]

def hand_rank(hand):
	score = list(zip(*sorted(((v, values[k]) for
		k,v in Counter(x[0] for x in hand).items()), reverse=True)))
	score[0] = ranks.index(score[0])
	if len(set(card[1] for card in hand)) == 1: score[0] = 5  # flush
	if score[1] in straights: score[0] = 8 if score[0] == 5 else 4  # str./str. flush
	return score

print ("P1 wins", sum(hand_rank(hand[:5]) > hand_rank(hand[5:]) for hand in hands))

P1 wins 376


In [8]:
cards1 = ['8C', '7S', '9H', '6D', 'TC']
cards2 = ['8C', '8S', '9H', '8D', '4C']
cards3 = ['8C', '8S', '9H', '9D', '4C']
cards4 = ['8C', '8S', '5H', '5D', '4C']
print(hand_rank(cards1),
      hand_rank(cards2),
      hand_rank(cards3),
      hand_rank(cards4))
hand_rank(cards1) > hand_rank(cards2)

[4, (10, 9, 8, 7, 6)] [3, (8, 9, 4)] [2, (9, 8, 4)] [2, (8, 5, 4)]


True

## Reference:
1. https://blog.dreamshire.com/project-euler-54-solution/